<p align="center">
  <img src="header.png" width="100%">
</p>


<div style="text-align: center;">
    <strong style="display: block; margin-bottom: 10px;">Group P</strong> 
    <table style="margin: 0 auto; border-collapse: collapse; border: 1px solid black;">
        <tr>
            <th style="border: 1px solid white; padding: 8px;">Name</th>
            <th style="border: 1px solid white; padding: 8px;">Student ID</th>
        </tr>
        <tr>
            <td style="border: 1px solid white; padding: 8px;">Beatriz Monteiro</td>
            <td style="border: 1px solid white; padding: 8px;">20240591</td>
        </tr>
        <tr>
            <td style="border: 1px solid white; padding: 8px;">Catarina Nunes</td>
            <td style="border: 1px solid white; padding: 8px;">20230083</td>
        </tr>
        <tr>
            <td style="border: 1px solid white; padding: 8px;">Margarida Raposo</td>
            <td style="border: 1px solid white; padding: 8px;">20241020</td>
        </tr>
        <tr>
            <td style="border: 1px solid white; padding: 8px;">Teresa Menezes</td>
            <td style="border: 1px solid white; padding: 8px;">20240333</td>
        </tr>
    </table>
</div>

### 🔗 Table of Contents <a id='table-of-contents'></a>
1. [Introduction](#introduction)  
2. [Macro's Prediction](#business-understanding)  
3. [Conclusion](#conclusion)

---

In [68]:
import os
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pmdarima import auto_arima
from statsmodels.tsa.arima.model import ARIMA
from scipy.stats import shapiro
from statsmodels.tsa.stattools import adfuller
from scipy.stats import norm
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import StandardScaler
from scipy.stats import spearmanr
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import shap
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [69]:
def load_dfs_from_folder(folder_path):
    """Loads DataFrames from files in a specified folder and returns a dictionary."""
    dfs = {}
    # List all files in the folder
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".pkl"):
            key = file_name.replace(".pkl", "")  # Extract key from the file name
            file_path = os.path.join(folder_path, file_name)
            
            # Load the dataframe from the pickle file
            with open(file_path, 'rb') as f:
                dfs[key] = pickle.load(f)
            print(f"Loaded {key} from {file_path}")
    
    return dfs

# Load both product_dfs and lagged_product_dfs from their respective folders
product_dfs = load_dfs_from_folder("product_dfs_folder")
lagged_product_dfs = load_dfs_from_folder("lagged_product_dfs_folder")

Loaded P11 from product_dfs_folder/P11.pkl
Loaded P13 from product_dfs_folder/P13.pkl
Loaded P12 from product_dfs_folder/P12.pkl
Loaded P16 from product_dfs_folder/P16.pkl
Loaded P14 from product_dfs_folder/P14.pkl
Loaded P8 from product_dfs_folder/P8.pkl
Loaded P9 from product_dfs_folder/P9.pkl
Loaded Sales_CPI from product_dfs_folder/Sales_CPI.pkl
Loaded P1 from product_dfs_folder/P1.pkl
Loaded P3 from product_dfs_folder/P3.pkl
Loaded P6 from product_dfs_folder/P6.pkl
Loaded P4 from product_dfs_folder/P4.pkl
Loaded P5 from product_dfs_folder/P5.pkl
Loaded P36 from product_dfs_folder/P36.pkl
Loaded P20 from product_dfs_folder/P20.pkl
Loaded P11 from lagged_product_dfs_folder/P11.pkl
Loaded P13 from lagged_product_dfs_folder/P13.pkl
Loaded P12 from lagged_product_dfs_folder/P12.pkl
Loaded P16 from lagged_product_dfs_folder/P16.pkl
Loaded P8 from lagged_product_dfs_folder/P8.pkl
Loaded P9 from lagged_product_dfs_folder/P9.pkl
Loaded P1 from lagged_product_dfs_folder/P1.pkl
Loaded P3 fro

In [70]:
for product_id in product_dfs.keys():
    product_dfs[product_id] = product_dfs[product_id].rename(columns={product_id: "Sales"})

for product_id in lagged_product_dfs.keys():
    lagged_product_dfs[product_id] = lagged_product_dfs[product_id].rename(columns={product_id: "Sales"})

In [71]:
product_dfs['P1'].head()

,Sales,MAB_ELE_SHP840,PRI27276_org,PRO27826_org,MAB_ELE_PRO276,MAB_ELE_SHP1100
month_year,,,,,,
2018-10-01,3.577403e+07,127.808839,109.119614,118.670791,124.227879,130.989253
2018-11-01,5.063649e+06,117.675874,109.224838,120.467019,127.404132,132.934130
2018-12-01,3.732127e+07,123.280134,109.330063,105.378705,120.518565,131.261348
2019-01-01,2.709040e+07,111.043755,109.750961,107.174933,104.776326,113.057565
2019-02-01,3.413209e+07,116.736921,109.856194,110.647640,109.597012,117.704727


In [72]:
product_ids = [1, 3, 4, 5, 6, 8, 9, 11, 12, 13, 14, 16, 20, 36]
macros_list = []

# Iterando sobre os IDs de produtos que você forneceu
for prod_id in product_ids:
    key = f'P{prod_id}'  # Construindo a chave do produto (ex: 'P1', 'P3', etc.)
    
    # Verificando se a chave do produto existe no dicionário
    if key in product_dfs:
        df = product_dfs[key]
        
        # Excluindo a coluna 'sales'
        df_without_sales = df.drop(columns=['Sales'])
        
        # Adicionando os dados de macros à lista
        macros_list.append(df_without_sales)

# Concatenando todos os DataFrames em uma única tabela
macros_combinations = pd.concat(macros_list, ignore_index=True)

# Obtendo todas as combinações únicas de macros
unique_combinations = macros_combinations.drop_duplicates()

# Exibindo as combinações únicas
unique_combinations

,MAB_ELE_SHP840,PRI27276_org,PRO27826_org,MAB_ELE_PRO276,MAB_ELE_SHP1100,PRO271000_org,PRI27840_org,PRO27380_org,WKLWEUR840_org,PRO27276_org,...,MAB_ELE_SHP276,MAB_ELE_PRO756,PRO27756_org,PRO27392_org,PRO28380_org,PRO28276_org,PRO28826_org,MAB_ELE_SHP250,PRO27250_org,PRO28392_org
0,127.808839,109.119614,118.670791,124.227879,130.989253,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,117.675874,109.224838,120.467019,127.404132,132.934130,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,123.280134,109.330063,105.378705,120.518565,131.261348,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,111.043755,109.750961,107.174933,104.776326,113.057565,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,116.736921,109.856194,110.647640,109.597012,117.704727,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,106.704029,114.326241,NaN,NaN,NaN,NaN,NaN,NaN,NaN
598,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,103.499260,108.999212,NaN,NaN,NaN,NaN,NaN,NaN,NaN
599,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,100.294492,103.672183,NaN,NaN,NaN,NaN,NaN,NaN,NaN
600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,97.089723,98.345154,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
# Lista dos números dos produtos que você mencionou
product_ids = [1, 3, 4, 5, 6, 8, 9, 11, 12, 13, 14, 16, 20, 36]

# Lista para armazenar os dados de macros de todos os produtos
macros_list = []

# Iterando sobre os IDs de produtos que você forneceu
for prod_id in product_ids:
    key = f'P{prod_id}'  # Construindo a chave do produto (ex: 'P1', 'P3', etc.)
    
    # Verificando se a chave do produto existe no dicionário
    if key in product_dfs:
        df = product_dfs[key]
        
        # Excluindo a coluna 'sales'
        df_sem_sales = df.drop(columns=['Sales'])
        
        # Adicionando os dados de macros à lista
        macros_list.append(df_sem_sales)

# Concatenando todos os DataFrames em uma única tabela
macros_combinados = pd.concat(macros_list, ignore_index=True)

# Obtendo todas as combinações únicas de macros
combinacoes_unicas = macros_combinados.drop_duplicates()

# Exibindo as combinações únicas
combinacoes_unicas


,MAB_ELE_SHP840,PRI27276_org,PRO27826_org,MAB_ELE_PRO276,MAB_ELE_SHP1100,PRO271000_org,PRI27840_org,PRO27380_org,WKLWEUR840_org,PRO27276_org,...,MAB_ELE_SHP276,MAB_ELE_PRO756,PRO27756_org,PRO27392_org,PRO28380_org,PRO28276_org,PRO28826_org,MAB_ELE_SHP250,PRO27250_org,PRO28392_org
0,127.808839,109.119614,118.670791,124.227879,130.989253,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,117.675874,109.224838,120.467019,127.404132,132.934130,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,123.280134,109.330063,105.378705,120.518565,131.261348,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,111.043755,109.750961,107.174933,104.776326,113.057565,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,116.736921,109.856194,110.647640,109.597012,117.704727,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,106.704029,114.326241,NaN,NaN,NaN,NaN,NaN,NaN,NaN
598,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,103.499260,108.999212,NaN,NaN,NaN,NaN,NaN,NaN,NaN
599,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,100.294492,103.672183,NaN,NaN,NaN,NaN,NaN,NaN,NaN
600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,97.089723,98.345154,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
# Lista dos números dos produtos que você mencionou
product_ids = [1, 3, 4, 5, 6, 8, 9, 11, 12, 13, 14, 16, 20, 36]

# Dicionário para armazenar as combinações únicas de macros e os produtos onde aparecem
combinacoes_produtos = {}

# Iterando sobre os IDs de produtos que você forneceu
for prod_id in product_ids:
    key = f'P{prod_id}'  # Construindo a chave do produto (ex: 'P1', 'P3', etc.)
    
    # Verificando se a chave do produto existe no dicionário
    if key in product_dfs:
        df = product_dfs[key]
        
        # Iterando sobre cada linha do DataFrame, considerando todas as colunas, exceto 'Sales'
        for idx, row in df.iterrows():
            # Excluindo a coluna 'Sales' apenas para a combinação de macros
            combinacao_macros = tuple(row.drop('Sales', errors='ignore'))  # Usando 'errors=ignore' para evitar erro se 'Sales' não existir
            
            # Adicionando o produto à lista de produtos para essa combinação de macros
            if combinacao_macros not in combinacoes_produtos:
                combinacoes_produtos[combinacao_macros] = []
            
            combinacoes_produtos[combinacao_macros].append(key)

# Convertendo o dicionário em um DataFrame para visualização
combinacoes_df = pd.DataFrame(list(combinacoes_produtos.items()), columns=['Combinação de Macros', 'Produtos'])
combinacoes_df['Produtos'] = combinacoes_df['Produtos'].apply(lambda x: ', '.join(x))  # Lista de produtos como string

# Exibindo o DataFrame final
combinacoes_df


,Combinação de Macros,Produtos
0,"(127.80883943812468, 109.1196136474609, 118.67...",P1
1,"(117.67587388309826, 109.2248382568359, 120.46...",P1
2,"(123.28013378640512, 109.3300628662109, 105.37...",P1
3,"(111.04375462185241, 109.7509613037109, 107.17...",P1
4,"(116.73692110885848, 109.8561935424805, 110.64...",P1
...,...,...
597,"(106.70402895817061, 114.32624113475178, 310.7...",P36
598,"(103.49926026575272, 108.99921197793539, 235.9...",P36
599,"(100.29449157333487, 103.672182821119, 235.956...",P36
600,"(97.08972288091698, 98.3451536643026, 329.4133...",P36


In [75]:
product_ids = [1, 3, 4, 5, 6, 8, 9, 11, 12, 13, 14, 16, 20, 36]


# Dicionário para armazenar as macros e os produtos onde elas aparecem
macros_produtos = {}

# Iterando sobre os IDs de produtos que você forneceu
for prod_id in product_ids:
    key = f'P{prod_id}'  # Construindo a chave do produto (ex: 'P1', 'P3', etc.)
    
    # Verificando se a chave do produto existe no dicionário
    if key in product_dfs:
        df = product_dfs[key]
        
        # Iterando sobre cada coluna de macros
        for macro in ['proteinas', 'carboidratos', 'gorduras']:
            # Se a macro estiver no DataFrame, vamos verificar se o valor existe
            if macro in df.columns:
                # Adicionando o produto à lista de produtos para essa macro
                for value in df[macro]:
                    if key not in macros_produtos[macro]:
                        macros_produtos[macro].append(key)

# Convertendo o dicionário em um DataFrame para visualização
macros_df = pd.DataFrame(list(macros_produtos.items()), columns=['Macro', 'Produtos'])
macros_df['Produtos'] = macros_df['Produtos'].apply(lambda x: ', '.join(x))  # Lista de produtos como string

# Exibindo o DataFrame final
print(macros_df)

Empty DataFrame
Columns: [Macro, Produtos]
Index: []


In [76]:
# Iterar sobre todos os DataFrames no dicionário `product_dfs` e exibir as colunas
for key, df in product_dfs.items():
    print(f"Colunas do DataFrame {key}:")
    print(df.columns)
    print()  # Apenas para separar as saídas de cada DataFrame


Colunas do DataFrame P11:
Index(['Sales', 'PRO27826_org', 'MAB_ELE_SHP392', 'MAB_ELE_SHP840',
       'MAB_ELE_SHP276'],
      dtype='object')

Colunas do DataFrame P13:
Index(['Sales', 'MAB_ELE_PRO756', 'PRO27756_org', 'MAB_ELE_PRO276',
       'PRI27840_org'],
      dtype='object')

Colunas do DataFrame P12:
Index(['Sales', 'PRI27840_org', 'RohCOPPER1000_org', 'MAB_ELE_PRO156'], dtype='object')

Colunas do DataFrame P16:
Index(['Sales', 'MAB_ELE_PRO756', 'PRO28276_org', 'PRI27276_org',
       'PRO28826_org'],
      dtype='object')

Colunas do DataFrame P14:
Index(['Sales', 'PRO27392_org', 'PRO28380_org', 'PRO27756_org'], dtype='object')

Colunas do DataFrame P8:
Index(['Sales', 'PRI27840_org', 'RohCOPPER1000_org'], dtype='object')

Colunas do DataFrame P9:
Index(['Sales', 'PRO27826_org', 'PRO271000_org', 'PRO28250_org',
       'MAB_ELE_PRO156'],
      dtype='object')

Colunas do DataFrame Sales_CPI:
Index(['Sales', 'MAB_ELE_SHP840', 'PRI27276_org', 'PRO271000_org',
       'PRO27826_org

In [77]:
# Lista de produtos para evitar na coluna 'Macro'
produtos = {"P1", "P3", "P4", "P5", "P6", "P8", "P9", "P11", "P12", "P13", "P14", "P16", "P20", "P36"}

# Dicionário de DataFrames com os dados dos produtos
product_dfs = {
    'P1': pd.read_pickle('product_dfs_folder/P1.pkl'),
    'P3': pd.read_pickle('product_dfs_folder/P3.pkl'),
    'P4': pd.read_pickle('product_dfs_folder/P4.pkl'),
    'P5': pd.read_pickle('product_dfs_folder/P5.pkl'),
    'P6': pd.read_pickle('product_dfs_folder/P6.pkl'),
    'P8': pd.read_pickle('product_dfs_folder/P8.pkl'),
    'P9': pd.read_pickle('product_dfs_folder/P9.pkl'),
    'P11': pd.read_pickle('product_dfs_folder/P11.pkl'),
    'P12': pd.read_pickle('product_dfs_folder/P12.pkl'),
    'P13': pd.read_pickle('product_dfs_folder/P13.pkl'),
    'P14': pd.read_pickle('product_dfs_folder/P14.pkl'),
    'P16': pd.read_pickle('product_dfs_folder/P16.pkl'),
    'P20': pd.read_pickle('product_dfs_folder/P20.pkl'),
    'P36': pd.read_pickle('product_dfs_folder/P36.pkl'),
}

# Dicionário para armazenar as macros e os produtos correspondentes
identificadores_produtos = {}

# Iterando sobre os DataFrames
for key, df in product_dfs.items():
    # Iterando sobre as colunas (exceto 'Sales')
    for col in df.columns:
        if col != 'Sales' and col not in produtos:  # Ignora 'Sales' e produtos na coluna 'Macro'
            if col not in identificadores_produtos:
                identificadores_produtos[col] = []
            identificadores_produtos[col].append(key)

# Convertendo o dicionário em um DataFrame para visualização
identificadores_df = pd.DataFrame(list(identificadores_produtos.items()), columns=['Macro', 'Produtos'])
identificadores_df['Produtos'] = identificadores_df['Produtos'].apply(lambda x: ', '.join(x))  # Lista de produtos como string

# Exibindo o DataFrame final
identificadores_df

,Macro,Produtos
0,MAB_ELE_SHP840,"P1, P5, P11"
1,PRI27276_org,"P1, P16"
2,PRO27826_org,"P1, P3, P5, P9, P11"
3,MAB_ELE_PRO276,"P1, P13"
4,MAB_ELE_SHP1100,P1
5,PRO271000_org,"P3, P5, P9, P20"
6,PRI27840_org,"P3, P8, P12, P13"
7,PRO27380_org,P4
8,WKLWEUR840_org,P4
9,PRO27276_org,"P4, P6"


---
##### <span style="background-color:#000027; padding:5px; border-radius:5px;">**Analyze Stationarity / Non-Stationarity per Macro**</span> <a id='Stationarity'></a>  

Click [here](#table-of-contents) ⬆️ to return to the Index.

In [82]:
from statsmodels.tsa.stattools import adfuller
import pandas as pd

# Função para aplicar o teste Dickey-Fuller (ADF) em uma série
def adf_test(series):
    """Realiza o teste Augmented Dickey-Fuller (ADF) para estacionariedade."""
    if isinstance(series, pd.Series):  # Verificar se a série é uma pd.Series
        series = series.dropna()  # Remover NaNs antes de realizar o teste
    if series.ndim == 1:  # Verificar se a série é unidimensional
        result = adfuller(series)  # Realiza o teste ADF
        print("Dickey-Fuller Test Results:")
        df_results = pd.Series(result[:4], index=["Test Statistic", "p-value", "# Lags Used", "# Observations Used"])
        for key, value in result[4].items():
            df_results[f"Critical Value ({key})"] = value
        print(df_results)
        
        # Interpretação
        if result[1] <= 0.05:
            print("✅ A série é estacionária (p-valor < 0.05).")
        else:
            print("❌ A série NÃO é estacionária (p-valor > 0.05). Considere transformar a série.")
    else:
        print("A série precisa ser unidimensional para o teste ADF.")

# Função para verificar a estacionariedade por macro
def check_stationarity_for_macros(identificadores_df, product_dfs):
    """Verifica a estacionariedade das variáveis macro combinando as séries dos produtos."""
    
    # Criar lista para armazenar os resultados
    stationarity_results = []

    # Iterar pelas macross de produtos
    for macro in identificadores_df["Macro"]:
        print(f"\nMacro: {macro}")
        
        # Identificar os produtos da macro
        produtos_macro = identificadores_df.loc[identificadores_df["Macro"] == macro, "Produtos"].values[0]
        produtos_lista = produtos_macro.split(", ")

        # Criar uma lista para as séries dos produtos
        combined_series = []

        # Iterar sobre os produtos e coletar as séries correspondentes
        for produto in produtos_lista:
            if produto in product_dfs:
                # Coletar a série temporal de cada produto, sem depender da coluna 'Sales'
                series = product_dfs[produto].dropna()  # Remover NaNs
                combined_series.append(series)

        # Concatenar as séries de todos os produtos da macro
        if combined_series:
            final_series = pd.concat(combined_series, axis=0)  # Concatenar todas as séries em uma única
            final_series = final_series.dropna()  # Remover NaNs da série concatenada
            final_series = final_series.squeeze()  # Garantir que a série seja unidimensional (squeeze)
            print(f"Testando estacionariedade da macro '{macro}'...")
            adf_test(final_series)  # Aplicar o teste ADF na série concatenada
            stationarity_results.append({"Macro": macro, "ADF P-Value": adfuller(final_series)[1]})

    # Criar um DataFrame com os resultados das análises
    stationarity_df = pd.DataFrame(stationarity_results)
    return stationarity_df

# Exemplo de execução:
stationarity_df = check_stationarity_for_macros(identificadores_df, product_dfs)

# Exibir os resultados
print("\nResultados da Estacionariedade por Macro:")
print(stationarity_df)



Macro: MAB_ELE_SHP840
Testando estacionariedade da macro 'MAB_ELE_SHP840'...
A série precisa ser unidimensional para o teste ADF.


ValueError: x is required to have ndim 1 but has ndim 2

In [66]:
# Função para aplicar o teste Dickey-Fuller (ADF) em uma série
def adf_test(series):
    """Realiza o teste Augmented Dickey-Fuller (ADF) para estacionariedade."""
    result = adfuller(series.dropna())  # Remover NaNs antes de realizar o teste
    print("Dickey-Fuller Test Results:")
    df_results = pd.Series(result[:4], index=["Test Statistic", "p-value", "# Lags Used", "# Observations Used"])
    for key, value in result[4].items():
        df_results[f"Critical Value ({key})"] = value
    print(df_results)
    
    # Interpretação
    if result[1] <= 0.05:
        print("✅ A série é estacionária (p-valor < 0.05).")
    else:
        print("❌ A série NÃO é estacionária (p-valor > 0.05). Considere transformar a série.")

# Função para verificar a estacionariedade por macro
def check_stationarity_for_macros(identificadores_df, product_dfs):
    """Verifica a estacionariedade das variáveis macro combinando as séries dos produtos."""
    
    # Criar lista para armazenar os resultados
    stationarity_results = []

    # Iterar pelas macross de produtos
    for macro in identificadores_df["Macro"]:
        print(f"\nMacro: {macro}")
        
        # Identificar os produtos da macro
        produtos_macro = identificadores_df.loc[identificadores_df["Macro"] == macro, "Produtos"].values[0]
        produtos_lista = produtos_macro.split(", ")

        # Criar uma lista para as séries dos produtos
        combined_series = []

        # Iterar sobre os produtos e coletar as séries correspondentes
        for produto in produtos_lista:
            if produto in product_dfs:
                # Verificar se a coluna 'Sales' existe no DataFrame do produto
                if 'Sales' in product_dfs[produto].columns:
                    series = product_dfs[produto]["Sales"].dropna()
                    combined_series.append(series)
                else:
                    print(f"❌ Coluna 'Sales' não encontrada no produto: {produto}")
                    continue  # Ignorar o produto se não tiver a coluna 'Sales'

        # Concatenar as séries de todos os produtos da macro
        if combined_series:
            final_series = pd.concat(combined_series)
            print(f"Testando estacionariedade da macro '{macro}'...")
            adf_test(final_series)  # Aplicar o teste ADF na série concatenada
            stationarity_results.append({"Macro": macro, "ADF P-Value": adfuller(final_series)[1]})

    # Criar um DataFrame com os resultados das análises
    stationarity_df = pd.DataFrame(stationarity_results)
    return stationarity_df

# Exemplo de execução:
stationarity_df = check_stationarity_for_macros(identificadores_df, product_dfs)

# Exibir os resultados
print("\nResultados da Estacionariedade por Macro:")
print(stationarity_df)


Macro: MAB_ELE_SHP840
❌ Coluna 'Sales' não encontrada no produto: P1
❌ Coluna 'Sales' não encontrada no produto: P5
❌ Coluna 'Sales' não encontrada no produto: P11

Macro: PRI27276_org
❌ Coluna 'Sales' não encontrada no produto: P1
❌ Coluna 'Sales' não encontrada no produto: P16

Macro: PRO27826_org
❌ Coluna 'Sales' não encontrada no produto: P1
❌ Coluna 'Sales' não encontrada no produto: P3
❌ Coluna 'Sales' não encontrada no produto: P5
❌ Coluna 'Sales' não encontrada no produto: P9
❌ Coluna 'Sales' não encontrada no produto: P11

Macro: MAB_ELE_PRO276
❌ Coluna 'Sales' não encontrada no produto: P1
❌ Coluna 'Sales' não encontrada no produto: P13

Macro: MAB_ELE_SHP1100
❌ Coluna 'Sales' não encontrada no produto: P1

Macro: PRO271000_org
❌ Coluna 'Sales' não encontrada no produto: P3
❌ Coluna 'Sales' não encontrada no produto: P5
❌ Coluna 'Sales' não encontrada no produto: P9
❌ Coluna 'Sales' não encontrada no produto: P20

Macro: PRI27840_org
❌ Coluna 'Sales' não encontrada no produt

In [67]:
# Função para aplicar o teste Dickey-Fuller (ADF) em uma série
def adf_test(series):
    """Realiza o teste Augmented Dickey-Fuller (ADF) para estacionariedade."""
    series = series.dropna()  # Remover NaNs antes de realizar o teste
    if series.ndim == 1:  # Verificar se a série é unidimensional
        result = adfuller(series)  # Realiza o teste ADF
        print("Dickey-Fuller Test Results:")
        df_results = pd.Series(result[:4], index=["Test Statistic", "p-value", "# Lags Used", "# Observations Used"])
        for key, value in result[4].items():
            df_results[f"Critical Value ({key})"] = value
        print(df_results)
        
        # Interpretação
        if result[1] <= 0.05:
            print("✅ A série é estacionária (p-valor < 0.05).")
        else:
            print("❌ A série NÃO é estacionária (p-valor > 0.05). Considere transformar a série.")
    else:
        print("A série precisa ser unidimensional para o teste ADF.")

# Função para verificar a estacionariedade por macro
def check_stationarity_for_macros(identificadores_df, product_dfs):
    """Verifica a estacionariedade das variáveis macro combinando as séries dos produtos."""
    
    # Criar lista para armazenar os resultados
    stationarity_results = []

    # Iterar pelas macross de produtos
    for macro in identificadores_df["Macro"]:
        print(f"\nMacro: {macro}")
        
        # Identificar os produtos da macro
        produtos_macro = identificadores_df.loc[identificadores_df["Macro"] == macro, "Produtos"].values[0]
        produtos_lista = produtos_macro.split(", ")

        # Criar uma lista para as séries dos produtos
        combined_series = []

        # Iterar sobre os produtos e coletar as séries correspondentes
        for produto in produtos_lista:
            if produto in product_dfs:
                # Coletar a série temporal de cada produto, sem depender da coluna 'Sales'
                series = product_dfs[produto].dropna()  # Remover NaNs
                combined_series.append(series)

        # Concatenar as séries de todos os produtos da macro
        if combined_series:
            final_series = pd.concat(combined_series, axis=0)  # Concatenar todas as séries em uma única (unidimensional)
            print(f"Testando estacionariedade da macro '{macro}'...")
            adf_test(final_series)  # Aplicar o teste ADF na série concatenada
            stationarity_results.append({"Macro": macro, "ADF P-Value": adfuller(final_series)[1]})

    # Criar um DataFrame com os resultados das análises
    stationarity_df = pd.DataFrame(stationarity_results)
    return stationarity_df

# Exemplo de execução:
stationarity_df = check_stationarity_for_macros(identificadores_df, product_dfs)

# Exibir os resultados
print("\nResultados da Estacionariedade por Macro:")
print(stationarity_df)



Macro: MAB_ELE_SHP840
Testando estacionariedade da macro 'MAB_ELE_SHP840'...
A série precisa ser unidimensional para o teste ADF.


ValueError: x is required to have ndim 1 but has ndim 2

###### <span style="background-color:#000027; padding:5px; border-radius:5px;font-size:13pt">**Autocorrelation Function (ACF)**</span> <a id='Autocorrelation'></a> 

In [83]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from statsmodels.tsa.stattools import acf, pacf

# Função para plotar a correlação para produtos, adaptado para previsão estacionária das macros
def plot_correlation_for_macros(identificadores_df, product_dfs, lags=30, plot_type='acf'):
    """
    Plots ACF or PACF for the combination of product series in each macro.
    
    Parameters:
    -----------
    identificadores_df : pd.DataFrame
        DataFrame contendo a relação de produtos e suas macros.
    product_dfs : dict
        Dicionário onde as chaves são os produtos e os valores são os DataFrames das séries temporais de cada produto.
    lags : int
        Número de lags para computar
    plot_type : str ('acf' ou 'pacf')
        Tipo de gráfico de correlação a ser exibido
    """
    # Validação do tipo de gráfico
    plot_type = plot_type.lower()
    if plot_type not in ['acf', 'pacf']:
        raise ValueError("plot_type must be either 'acf' or 'pacf'")

    # Inicializa o gráfico de subplots
    fig = make_subplots(
        rows=1,
        cols=len(identificadores_df["Macro"].unique()),  # Número de colunas igual ao número de macros únicas
        subplot_titles=[f"{plot_type.upper()}: {macro}" for macro in identificadores_df["Macro"].unique()],
        horizontal_spacing=0.01)

    # Itera sobre cada macro
    for i, macro in enumerate(identificadores_df["Macro"].unique()):
        print(f"\nAnalisando a macro: {macro}")
        
        # Identifica os produtos da macro
        produtos_macro = identificadores_df.loc[identificadores_df["Macro"] == macro, "Produtos"].values[0]
        produtos_lista = produtos_macro.split(", ")

        # Criar uma lista para as séries dos produtos
        combined_series = []

        # Itera sobre os produtos e coleta as séries temporais correspondentes
        for produto in produtos_lista:
            if produto in product_dfs:
                series = product_dfs[produto].dropna()  # Remove NaNs
                combined_series.append(series)

        # Concatenar as séries de todos os produtos da macro
        if combined_series:
            final_series = pd.concat(combined_series, axis=0)  # Concatena todas as séries em uma única série
            final_series = final_series.dropna()  # Remove NaNs da série concatenada
            
            if len(final_series) > 1:  # Garante que a série tenha mais de um ponto
                print(f"Testando estacionariedade da macro '{macro}'...")
                if plot_type == 'acf':
                    corr_values = acf(final_series, nlags=lags, fft=True)
                    # Erro padrão para ACF (fórmula de Bartlett)
                    se = 2.24 / np.sqrt(len(final_series))
                else:
                    corr_values = pacf(final_series, nlags=lags)
                    # Erro padrão para PACF (1/sqrt(n))
                    se = 2.24 / np.sqrt(len(final_series))
                
                lags_range = list(range(lags+1))
                
                # Cria o efeito "stem" (linha + marcador no topo)
                fig.add_trace(
                    go.Scatter(
                        x=lags_range, y=corr_values, mode='lines+markers', line=dict(color='blue', width=1),
                        marker=dict(color='blue', size=5), name=macro,
                        hovertemplate=f"Lag %{{x}}<br>{plot_type.upper()}: %{{y:.2f}}<extra></extra>"),
                    row=1, col=i+1)
                
                # Adiciona linhas verticais finas de 0 até o valor da correlação (efeito stem)
                for lag, val in zip(lags_range, corr_values):
                    fig.add_trace(
                        go.Scatter(
                            x=[lag, lag], y=[0, val], mode='lines', line=dict(color='blue', width=1),
                            showlegend=False, hoverinfo='skip'), row=1, col=i+1)
                
                # Adiciona intervalos de confiança constantes
                fig.add_hline(y=se, line=dict(color='red', width=1, dash='dash'), 
                             row=1, col=i+1, opacity=0.7)
                fig.add_hline(y=-se, line=dict(color='red', width=1, dash='dash'), 
                             row=1, col=i+1, opacity=0.7)
                
                # Área sombreada para o intervalo de confiança
                fig.add_hrect(y0=-se, y1=se, 
                             line_width=0, fillcolor='rgba(100,100,100,0.2)', 
                             row=1, col=i+1)

    # Atualiza o layout do gráfico
    fig.update_layout(
        title_text=f"{plot_type.upper()} por Macro (97.5% Confidence)",
        height=500,
        width=500*len(identificadores_df["Macro"].unique()),  # Ajusta o tamanho com base no número de macros
        showlegend=False,
        margin=dict(l=20, r=20))
    
    # Configurações uniformes dos eixos
    fig.update_yaxes(range=[-1.1, 1.1], title_text=plot_type.upper())
    fig.update_xaxes(title_text="Lag")
    
    # Exibe o gráfico
    fig.show()

# Exemplo de execução:
# Assumindo que 'identificadores_df' e 'product_dfs' já estão definidos, podemos chamar a função para plotar as correlações.
plot_correlation_for_macros(identificadores_df, product_dfs, lags=30, plot_type='acf')



Analisando a macro: MAB_ELE_SHP840

Analisando a macro: PRI27276_org

Analisando a macro: PRO27826_org

Analisando a macro: MAB_ELE_PRO276

Analisando a macro: MAB_ELE_SHP1100
Testando estacionariedade da macro 'MAB_ELE_SHP1100'...


ValueError: x is required to have ndim 1 but has ndim 2

In [ ]:
plot_correlation_for_products(df_sales, plot_type='acf')

###### <span style="background-color:#000027; padding:5px; border-radius:5px;font-size:13pt">**Partial Autocorrelation Function (PACF)**</span> <a id='PACF'></a> 

In [44]:

# Função para classificar as variáveis
def classify_variable(series):
    """Classifica a variável com base em testes de normalidade e estacionariedade."""
    
    clean_series = series.dropna()
    if len(clean_series) > 3:
        stat, p_value = shapiro(clean_series)
        is_normal = p_value > 0.05  # p-value > 0.05 significa normal
        
        adf_stat, adf_p_value, _, _, _, _ = adfuller(clean_series)
        is_stationary = adf_p_value < 0.05  # p-value < 0.05 significa estacionária
    else:
        is_normal = False
        is_stationary = False

    return is_normal, is_stationary, adf_p_value

# Aplicar análise de estacionariedade por macro
stationarity_results = []

for macro in identificadores_df["Macro"]:
    # Criar uma série temporal composta por produtos da macro
    produtos_macro = identificadores_df.loc[identificadores_df["Macro"] == macro, "Produtos"].values[0]
    produtos_lista = produtos_macro.split(", ")

    combined_series = []
    
    for produto in produtos_lista:
        if produto in product_dfs:
            series = product_dfs[produto]["Sales"].dropna()
            combined_series.append(series)

    # Concatenar todas as séries da macro
    if combined_series:
        final_series = pd.concat(combined_series)
        _, _, adf_p_value = classify_variable(final_series)
        stationarity_results.append({"Macro": macro, "ADF P-Value": adf_p_value})

# Criar DataFrame com os resultados da análise de estacionariedade
stationarity_df = pd.DataFrame(stationarity_results)

# Função de imputação automática
def auto_impute_missing_values(df_train, df_test):
    """Preenche valores ausentes automaticamente usando métodos adequados."""

    missing_columns = df_test.columns[df_test.isnull().any()]
    
    for col in missing_columns:
        print(f"Processando: {col}")
        series = df_train[col]  # Usar dados de treino para imputação

        is_normal, is_stationary, _ = classify_variable(series)

        if is_normal:
            print(f" - {col} é normal → Usando Média & Desvio Padrão")
            mean_value, std_value = series.mean(), series.std()
            num_missing = df_test[col].isnull().sum()
            predictions = norm.rvs(loc=mean_value, scale=std_value, size=num_missing)
        
        elif is_stationary:
            print(f" - {col} é estacionária → Usando Suavização Exponencial Simples")
            model = SimpleExpSmoothing(series.dropna()).fit()
            predictions = model.forecast(steps=df_test[col].isnull().sum())

        else:
            print(f" - {col} é não-estacionária → Usando ARIMA")
            model = ARIMA(series.dropna(), order=(1, 1, 1))
            fitted_model = model.fit()
            predictions = fitted_model.forecast(steps=df_test[col].isnull().sum())

        missing_indexes = df_test[df_test[col].isnull()].index
        df_test.loc[missing_indexes, col] = predictions

    return df_test

# Aplicando imputação automática no conjunto de teste
df_train = remerged_data[1]  
df_test = test_1.copy()
df_test_filled = auto_impute_missing_values(df_train, df_test)

# Exibir resultados finais
print("Análise de Estacionariedade por Macro:")
print(stationarity_df)
print("\nAmostra do Conjunto de Teste após Imputação:")
print(df_test_filled.head())


KeyError: 'Sales'

In [53]:
import pandas as pd
from scipy.stats import shapiro
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.arima.model import ARIMA
from scipy.stats import norm

# Função para classificar as variáveis
def classify_variable(series):
    """Classifica a variável com base em testes de normalidade e estacionariedade."""
    
    clean_series = series.dropna()
    if len(clean_series) > 3:
        stat, p_value = shapiro(clean_series)
        is_normal = p_value > 0.05  # p-value > 0.05 significa normal
        
        adf_stat, adf_p_value, _, _, _, _ = adfuller(clean_series)
        is_stationary = adf_p_value < 0.05  # p-value < 0.05 significa estacionária
    else:
        is_normal = False
        is_stationary = False
        adf_p_value = None  # Valor nulo se não houver dados suficientes

    return is_normal, is_stationary, adf_p_value

# Aplicar análise de estacionariedade por macro
stationarity_results = []

for macro in identificadores_df["Macro"]:
    # Criar uma série temporal composta por produtos da macro
    produtos_macro = identificadores_df.loc[identificadores_df["Macro"] == macro, "Produtos"].values[0]
    produtos_lista = produtos_macro.split(", ")

    combined_series = []
    
    for produto in produtos_lista:
        if produto in product_dfs:
            df = product_dfs[produto]

            # Escolher a primeira coluna numérica disponível para análise
            numeric_cols = df.select_dtypes(include=["number"]).columns
            if not numeric_cols.empty:
                series = df[numeric_cols[0]].dropna()
                combined_series.append(series)

    # Concatenar todas as séries da macro
    if combined_series:
        final_series = pd.concat(combined_series)
        _, _, adf_p_value = classify_variable(final_series)
        stationarity_results.append({"Macro": macro, "ADF P-Value": adf_p_value})

# Criar DataFrame com os resultados da análise de estacionariedade
stationarity_df = pd.DataFrame(stationarity_results)

# Função de imputação automática
def auto_impute_missing_values(df_train, df_test):
    """Preenche valores ausentes automaticamente usando métodos adequados."""

    missing_columns = df_test.columns[df_test.isnull().any()]
    
    for col in missing_columns:
        print(f"Processando: {col}")
        
        # Verifica se a coluna existe no conjunto de treino
        if col not in df_train.columns:
            print(f" - {col} não está presente no conjunto de treino. Pulando...")
            continue

        series = df_train[col].dropna()  # Remover valores ausentes

        if len(series) < 3:
            print(f" - {col} tem poucos dados para análise. Pulando...")
            continue

        is_normal, is_stationary, _ = classify_variable(series)

        if is_normal:
            print(f" - {col} é normal → Usando Média & Desvio Padrão")
            mean_value, std_value = series.mean(), series.std()
            num_missing = df_test[col].isnull().sum()
            predictions = norm.rvs(loc=mean_value, scale=std_value, size=num_missing)
        
        elif is_stationary:
            print(f" - {col} é estacionária → Usando Suavização Exponencial Simples")
            model = SimpleExpSmoothing(series).fit()
            predictions = model.forecast(steps=df_test[col].isnull().sum())

        else:
            print(f" - {col} é não-estacionária → Usando ARIMA")
            model = ARIMA(series, order=(1, 1, 1))
            fitted_model = model.fit()
            predictions = fitted_model.forecast(steps=df_test[col].isnull().sum())

        # Preencher valores ausentes
        missing_indexes = df_test[df_test[col].isnull()].index
        df_test.loc[missing_indexes, col] = predictions

    return df_test

# Aplicando imputação automática no conjunto de teste
df_train = remerged_data[1]  
df_test = test_1.copy()
df_test_filled = auto_impute_missing_values(df_train, df_test)

# Exibir resultados finais
print("Análise de Estacionariedade por Macro:")
print(stationarity_df)
print("\nAmostra do Conjunto de Teste após Imputação:")
print(df_test_filled.head())


NameError: name 'remerged_data' is not defined

### <span style="background-color:#000027; padding:5px; border-radius:5px;"> 📌 Prediction for the Macro Features used <a id='business-understanding'></a>

##### Click [here](#table-of-contents) ⬆️ to return to the Index.
---

In [ ]:
# Define the function to classify variables
def classify_variable(series):
    """Classifies variable based on normality and stationarity tests."""
    
    # Remove NaN values for testing
    clean_series = series.dropna()

    # Check for normality
    if len(clean_series) > 3:
        stat, p_value = shapiro(clean_series)
        is_normal = p_value > 0.05  # p-value > 0.05 means normal
    else:
        is_normal = False  # Not enough data to test normality

    # Check for stationarity
    if len(clean_series) > 3:
        adf_stat, adf_p_value, _, _, _, _ = adfuller(clean_series)
        is_stationary = adf_p_value < 0.05  # p-value < 0.05 means stationary
    else:
        is_stationary = False  # Not enough data

    return is_normal, is_stationary

# Function to automatically fill missing values
def auto_impute_missing_values(df_train, df_test):
    """Automatically selects the best imputation method for each missing variable."""
    
    # Identify missing columns in test set
    missing_columns = df_test.columns[df_test.isnull().any()]
    
    # Iterate through missing columns
    for col in missing_columns:
        print(f"Processing: {col}")

        series = df_train[col]  # Use train data for imputation
        is_normal, is_stationary = classify_variable(series)

        if is_normal:
            # Case 1: Normally distributed → Sample from normal distribution
            print(f" - {col} is normal → Using Mean & Std Sampling")
            mean_value, std_value = series.mean(), series.std()
            num_missing = df_test[col].isnull().sum()
            predictions = norm.rvs(loc=mean_value, scale=std_value, size=num_missing)
        
        elif is_stationary:
            # Case 2: Stationary but non-normal → Simple Exponential Smoothing
            print(f" - {col} is stationary → Using Simple Exponential Smoothing")
            model = SimpleExpSmoothing(series.dropna()).fit()
            predictions = model.forecast(steps=df_test[col].isnull().sum())

        elif not is_stationary:
            # Case 3: Non-Stationary → ARIMA
            print(f" - {col} is non-stationary → Using ARIMA")
            model = ARIMA(series.dropna(), order=(1, 1, 1))  # (p,d,q) chosen based on domain knowledge
            fitted_model = model.fit()
            predictions = fitted_model.forecast(steps=df_test[col].isnull().sum())

        else:
            # Case 4: If nothing works → Use XGBoost Regression
            print(f" - {col} is complex → Using XGBoost Regression")
            train_data = df_train.dropna(subset=[col])  # Drop missing values for training
            X_train = train_data.drop(columns=[col])  # Exclude target column
            y_train = train_data[col]  # Target column

            X_test = df_test.loc[df_test[col].isnull(), X_train.columns]  # Only missing values

            model = XGBRegressor(n_estimators=100, learning_rate=0.1)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)

        # Assign predictions
        missing_indexes = df_test[df_test[col].isnull()].index
        df_test.loc[missing_indexes, col] = predictions

    return df_test

# Example usage
df_train = remerged_data[1]  # Use remerged train data
df_test = test_1.copy()  # Copy test set

# Apply automatic imputation
df_test_filled = auto_impute_missing_values(df_train, df_test)

# Check results
print(df_test_filled.head())